In [1]:
#IMPORT LIBRARIES
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#API CALL FUNCTION
def replace_value(dict, value_to_find, value_to_replace):
    # This function is to replace specified value('\xa0') in dictionary
    for k, v in dict.items():
        for k1, v1 in v.items():
            if v1 == value_to_find:
                v[k1] = value_to_replace
            dict[k] = v
        return dict

def dict2df(dict, cols):
    raw_df = pd.DataFrame.from_dict(dict)
    df = raw_df.drop(raw_df.index[0])  # first row is duplicated with columns
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
    return df

def getByKeyword(id,keyword):
    url='http://hkgaet105/project/api/'+str(id)+'/'+keyword
    r = requests.get(url)
    #print (r.json()[0])
    value=r.json()[0]["json"]
    raw_dict=replace_value(value,'\xa0',"")
    cols=list(raw_dict.keys())
    df=dict2df(raw_dict,cols)
    return df

In [5]:
#SECTION 5: Input Parmeters Summary Table: Building Envelope
#This function assigns the 'type' of each entry in the dataframe. Maybe the boundaries can be discusses (can a roof be <180 degrees??)
def assignCategory(row):
    if row == 0:
        return "floor"
    elif row > 0 and row < 180:
        return "wall"
    elif row == 180:
        return "roof"

def assignType(row, types):
    i = 1
    for x in types:
        if row == x:
            return i
        else:
            i=i+1
            
def assignCatAndType(values):
    values["category"]=values["Tilt [deg]"].apply(assignCategory)
    #WALLS
    wall = values[(values.category=="wall")]        
    wall_u_types = wall["U-Factor with Film [W/m2-K]"].unique() #Gets list of unique U-values
    wall["type"] = wall["U-Factor with Film [W/m2-K]"].apply(assignType, types = wall_u_types) #Assigns a type to each row
    #ROOFS
    roof = values[(values.category=="roof")]        
    roof_u_types = roof["U-Factor with Film [W/m2-K]"].unique() #Gets list of unique U-values
    roof["type"] = roof["U-Factor with Film [W/m2-K]"].apply(assignType, types = roof_u_types) #Assigns a type to each row
    #FLOORS
    floor = values[(values.category=="floor")]        
    floor_u_types = floor["U-Factor with Film [W/m2-K]"].unique() #Gets list of unique U-values
    floor["type"] = floor["U-Factor with Film [W/m2-K]"].apply(assignType, types = floor_u_types) #Assigns a type to each row
    formattedvalues=pd.concat([wall,roof,floor], axis=0)
    return formattedvalues

    #For ROOF/WALL/FLOORS above, can groupby separately for each category, THEN concatenate 

    
def buildingEnvelope(id):
    values = getByKeyword(id,"opaque")
    formattedvalues = assignCatAndType(values)
    #USE GROUPBY (SEE BELOW) TO GROUP
    formattedvalues = formattedvalues.groupby(["type"])[["U-Factor no Film [W/m2-K]"]].mean()#[["Net Area [m2]"]].sum()
    print("hi")
    return formattedvalues
    

buildingEnvelope(2159)




hi


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

,U-Factor no Film [W/m2-K]
type,
1,0.502761
